In [1]:
! pip install datasets transformers
! pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [3]:
songs = pd.read_csv("combined_songs.csv")
# songs = songs[:10000]

songs_data = pd.DataFrame()

for genre in songs['genre'].unique():
    genre_subset = songs[songs['genre'] == genre]
    sampled_subset = genre_subset.sample(n=1250, random_state=1)
    songs_data = pd.concat([songs_data, sampled_subset])

songs_data.reset_index(drop=True, inplace=True)


In [4]:
"R&B".lower()

'r&b'

In [5]:
songs_data['genre'].unique()
labels_dict = {'rock':0, 'country':1, 'r&b':2, 'rap & hip hop':3, 'pop':4, 'rap':5}
labels_dict

{'rock': 0, 'country': 1, 'r&b': 2, 'rap & hip hop': 3, 'pop': 4, 'rap': 5}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 6)

pipe = pipeline("text-classification", model="bert-base-uncased", tokenizer = "bert-base-uncased" )

# def get_response(prompt: str) -> None:
#     """
#     Generate a response from the GPT2 model.

#     Parameters:
#         prompt (str): The user's input/question for the model.

#     Returns:
#         None: Prints the model's response.
#     """
#     sequences = pipe(
#         prompt,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         max_length=256,
#         truncation=True,
#     )
#     print("Chatbot:", sequences[0]['generated_text'])

train_data, test_data = train_test_split(songs_data, test_size = 0.2, random_state = 69)

train_data_dict = { 'genre': [str(ele["genre"]).lower() if not pd.isnull(ele["genre"]) else '' for _, ele in train_data.iterrows()]
                    , 'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in train_data.iterrows()]
                   }
train_data = Dataset.from_dict(train_data_dict)

test_data_dict = {'genre': [str(ele["genre"]).lower() if not pd.isnull(ele["genre"]) else '' for _, ele in test_data.iterrows()]
                  , 'lyrics': [str(ele["lyrics"]) if not pd.isnull(ele["lyrics"]) else '' for _, ele in test_data.iterrows()]
                  }
test_data = Dataset.from_dict(test_data_dict)

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["lyrics"], max_length=40, padding="max_length", truncation=True)
    tokenized_inputs["labels"] = [labels_dict[label] for label in examples["genre"]]
    return tokenized_inputs

lm_train_data = train_data.map(tokenize_function, batched=True, num_proc=4)#, remove_columns=train_data.column_names)
lm_test_data = test_data.map(tokenize_function, batched=True, num_proc=4)#, remove_columns=test_data.column_names)

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
# train_dataset = lm_train_data.rename_column("genre", "labels")
# test_dataset = lm_test_data.rename_column("genre", "labels")
lm_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
lm_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted'),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted'),
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_train_data,
    eval_dataset=lm_test_data,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
lm_train_data[0]

{'input_ids': tensor([  101,  4931,  1010,  2821,  8670,  1011,  2079,  1011,  8670,  1010,
          8670,  1011,  2079,  1011,  8670,  4931,  1010,  2821,  8670,  1011,
          2079,  1011,  8670,  1010,  8670,  1011,  2079,  1011,  8670,  3336,
          1010,  2202,  2125,  2115,  4658,  1045, 10587,  2156,  2017,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor(5)}

In [11]:
trainer.train()
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.2f}")
print(f"F1: {eval_results['eval_f1']:.2f}")
print(f"Precision: {eval_results['eval_precision']:.2f}")
print(f"Recall: {eval_results['eval_recall']:.2f}")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.989300,0.927561,0.617500,0.611886,0.623818,0.617500
2,0.691700,0.847805,0.667000,0.662936,0.672885,0.667000
3,0.419800,0.900482,0.689000,0.689047,0.689768,0.689000


Accuracy: 0.69
F1: 0.69
Precision: 0.69
Recall: 0.69


In [13]:
input_lyric = """
When I get on the dance floor with this girl who is a supermodel, I'm like, "Whoa!" It's like, you know, this isn't what she wants to do and she has to do this. And you can't be a good dancer.

You know, there are two things that I'm going out to do. One is to do a video on Instagram. It's not about the video. It's about the video. So what I've done is I have a video and a video and an image. So you can be a good dancer and be a good dancer but you'll have to learn how to do it. So that's why I'm in there doing this and I'm doing this.

So that's where the video comes into play.

Yeah, I'm a big, strong girl who's a great dancer. And I don't want you to be a good dancer and you won't be a good dancer. I'll tell ya what, I don't want to be a bad dancer. I'll tell ya what, I want to be great because I'm a big dancer. I'm a girl who wants to be great. But I'm a big, strong girl. I'm just a
"""


In [15]:
result = pipe(input_lyric)
print("Predicted Genre:", result[0]['label'])
print("Score:", result[0]['score'])

Predicted Genre: LABEL_1
Score: 0.5097446441650391
